In [235]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np

<img src = './docs/1.png' height ='800px'>

In [236]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

#Create a ansatz V with n qubits
def V(num_qubits: int):
    return ansatz.stargraph (num_qubits=num_qubits)

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calRho3 (rho, unitary_matrix, kraus_operators):
    rho2 = sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    rho3 = (np.transpose(np.conjugate(unitary_matrix)) @ rho2.data @ unitary_matrix)
    return rho3

def createKraus(unitary_matrix):
    kraus_ops = []
    Q, R = qr(unitary_matrix) #(16,16) -> vector 1x16

    #print(Q.shape)
    for q in Q:
        
        q = np.expand_dims(q, 1)
        #print(q)
        #print(np.transpose(np.conjugate(q)))
        #print(q @ np.transpose(np.conjugate(q)))
        kraus_ops.append(q @ np.transpose(np.conjugate(q))) #vector 1x16

    
    return kraus_ops



In [237]:
num_qubits = 2

#Create a ansatz V with n qubits
circuit = V(num_qubits)

#Assign random parameter
num_params = circuit.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)
circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))

print(circuit)

     ┌────────────┐   
q_0: ┤ Ry(1.5728) ├─■─
     ├────────────┤ │ 
q_1: ┤ Ry(2.0352) ├─■─
     └────────────┘   


In [238]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data
print (unitary_matrix.shape) #(16, 16 as num_qubits is 4)

(4, 4)


In [239]:
KrausOperators = createKraus(unitary_matrix=unitary_matrix)
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))


[[ 1.00000000e+00+0.j -3.33066907e-16+0.j -8.32667268e-17+0.j
   8.32667268e-17+0.j]
 [-3.33066907e-16+0.j  1.00000000e+00+0.j  5.55111512e-17+0.j
   5.55111512e-17+0.j]
 [-8.32667268e-17+0.j  5.55111512e-17+0.j  1.00000000e+00+0.j
  -2.22044605e-16+0.j]
 [ 8.32667268e-17+0.j  5.55111512e-17+0.j -2.22044605e-16+0.j
   1.00000000e+00+0.j]]


In [240]:
#rho
# rho = DensityMatrix.from_label('0' * 2)
# rho2 = Epsilon(rho=rho, kraus_operators=KrausOperators)
# print(rho2)

# rho3 = Epsilon2(rho=rho2, unitary_matrix=unitary_matrix)
# print(rho3)


In [241]:
#rho
rho = DensityMatrix.from_label('0' * num_qubits)
rho3 = calRho3(rho=rho, kraus_operators=KrausOperators, unitary_matrix=unitary_matrix)
print(rho3)


[[ 0.22722955+0.j -0.08951331+0.j -0.04520413+0.j  0.04464388+0.j]
 [-0.08951331+0.j  0.22783867+0.j  0.04504685+0.j -0.04448497+0.j]
 [-0.04520413+0.j  0.04504685+0.j  0.27297232+0.j  0.08951088+0.j]
 [ 0.04464388+0.j -0.04448497+0.j  0.08951088+0.j  0.27195945+0.j]]


In [242]:
#cost func compare rho, rho3
def cost(rho, rho3):
    return 1-metric.compilation_trace_fidelity(rho, rho3)

print(cost(rho=rho, rho3=rho3))

#Nhận U, K trả về rho3

0.536859190970439


In [243]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

<img src = './docs/2.png' height ='800px'>

<img src = './docs/3.jpg' height ='800px'>

In [278]:
def Derivative(rho, unitary_matrix, kraus_operators, epsilon=0.01, alpha=0.1):
    kraus_operators_plus = kraus_operators.copy()
    kraus_operators_minus = kraus_operators.copy()
    c = []
    
    # Loop through Kraus operators
    for n, k in enumerate(kraus_operators):
        k_plus = k + epsilon # Kj(+epsilon)
        k_minus = k - epsilon # Kj(-epsilon)
      
        kraus_operators_plus[n] = k_plus # Replace Kj by new Kj_plus
        kraus_operators_minus[n] = k_minus # Replace Kj by new Kj_minus
        #print(kraus_operators_minus)
        #print(kraus_operators_plus)

        # Compute fidelity for K+ and K-
        fil_plus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_plus))
        fil_minus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_minus))
        
        # Compute the derivative
        derivative = -(fil_plus - fil_minus) / (epsilon)

        # Accumulate
        c.append(derivative)
        
        # Reset Kraus operators for next iteration
        kraus_operators_plus[n] = k
        kraus_operators_minus[n] = k
    
    # Update Kraus operators
    kraus_operators_res = [k - c[n]*alpha for n,k in enumerate(kraus_operators)]
    print ('c=', c)
    #print(kraus_operators_res)
    return kraus_operators_res

In [279]:
# Create a copy
KrausOperatorsTry = KrausOperators.copy()
a = 1

# Loop until the cost <= 0.01
while (a > 0.0000000001):
    
    KrausOperatorsTry = Derivative(rho, unitary_matrix, KrausOperatorsTry) # Update Kraus Operators 
    print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # Should be I though
    a = cost(rho, calRho3(rho, unitary_matrix, KrausOperatorsTry))
    print (a)

c= [0.7758817144094987, -0.29600465569620815, -0.6342603069825481, -0.6363337990004936]
[[1.1987421 +0.j 0.19877849+0.j 0.18354637+0.j 0.18348633+0.j]
 [0.19877849+0.j 1.19881489+0.j 0.18358277+0.j 0.18352273+0.j]
 [0.18354637+0.j 0.18358277+0.j 1.16835064+0.j 0.1682906 +0.j]
 [0.18348633+0.j 0.18352273+0.j 0.1682906 +0.j 1.16823057+0.j]]
0.4716318788386432
c= [0.5729098869502125, -0.22646084193573834, -0.5108682047920143, -0.46610875967827425]
[[1.42500519+0.j 0.42507964+0.j 0.39911325+0.j 0.39901517+0.j]
 [0.42507964+0.j 1.4251541 +0.j 0.3991877 +0.j 0.39908963+0.j]
 [0.39911325+0.j 0.3991877 +0.j 1.37322131+0.j 0.37312323+0.j]
 [0.39901517+0.j 0.39908963+0.j 0.37312323+0.j 1.37302516+0.j]]
0.43256213740678484
c= [0.45214171920193724, -0.17158705533671936, -0.43549306654748765, -0.4151439655717404]
[[1.67264101+0.j 0.67274172+0.j 0.63586875+0.j 0.63573888+0.j]
 [0.67274172+0.j 1.67284243+0.j 0.63596947+0.j 0.6358396 +0.j]
 [0.63586875+0.j 0.63596947+0.j 1.5990965 +0.j 0.59896663+0.j]

In [280]:
KrausOperatorsTry = KrausOperators.copy()

In [309]:
# try looping manually
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry))
a = cost(rho, calRho3(rho, unitary_matrix, KrausOperatorsTry))
KrausOperatorsTry = Derivative(rho, unitary_matrix, KrausOperatorsTry) 
print (a)

[[15.34870412+0.j 14.34828896+0.j 13.89259569+0.j 13.89159507+0.j]
 [14.34828896+0.j 15.3478738 +0.j 13.89218053+0.j 13.89117991+0.j]
 [13.89259569+0.j 13.89218053+0.j 14.43648727+0.j 13.43548664+0.j]
 [13.89159507+0.j 13.89117991+0.j 13.43548664+0.j 14.43448601+0.j]]
c= [0.20760755305622158, -0.027571016171701324, -0.2711538693094928, -0.43347839568068647]
-0.002683252607695197
